***

<font size="500">Precipitation Analysis</font>

***

# Libraries

In [ ]:
# pip install --user --upgrade --force-reinstall "git+https://github.com/joaquinseguraellis/PydroDesign.git"

In [ ]:
from RiSA.basics import *
from RiSA.download import *
from RiSA.frequency_analysis import *
from RiSA.geo_tools import *
from RiSA.hydro_tools import *
from RiSA.interp import *
from RiSA.masks import *
from RiSA.figures import *
from RiSA.results import *
from RiSA.rg_pre import *
from RiSA import __version__
print(f'RiSA version: {__version__.version()}')

download_credentials(
    urs='urs.earthdata.nasa.gov',
    username='joako19',
    password='$Q#9mqZ%)ZV5jDt'
)

# Main

In [ ]:
class Main:
    """
    Main execution class for the project
    """

    def __init__(self):
        self.start_date = datetime.datetime(2000, 7, 1)
        self.end_date   = datetime.datetime(2021, 7, 1)
        self.bbox = [-76.05, -53.05, -55.95, -21.05] # lon1, lon2, lat1, lat2
        self.lon_IMERG = np.arange(self.bbox[0], self.bbox[1] + 0.001, 0.1)
        self.lat_IMERG = np.arange(self.bbox[2], self.bbox[3] + 0.001, 0.1)
        self.period = ['Y', 'jja', 'son', 'djf', 'mam']
        self.T = np.arange(2, 50.1, 1)
        self.error = list()
        self.stations = list()
        self.test_w_out = list()
        self.test = list()
        self.for_use_w_out = list()
        self.for_use = list()

    def create_paths(
            self, stations_dir, rg_bin_dir, imerg_bin_dir,
    ):
        """
        
        """
        self.stations_dir = stations_dir
        self.outputs_dir = Path('outputs')
        if not os.path.exists(self.outputs_dir):
            os.mkdir(self.outputs_dir)
        self.card_dir = Path(self.outputs_dir, f'cards_Y_rx1day')
        if not os.path.exists(self.card_dir):
            os.mkdir(self.card_dir)
        self.bin_dir = Path('bin')
        if not os.path.exists(self.bin_dir):
            os.mkdir(self.bin_dir)
        self.rg_bin_dir = Path(self.bin_dir, rg_bin_dir)
        if not os.path.exists(self.rg_bin_dir):
            os.mkdir(self.rg_bin_dir)
        self.imerg_bin_dir = Path(self.bin_dir, imerg_bin_dir)
        if not os.path.exists(self.imerg_bin_dir):
            os.mkdir(self.imerg_bin_dir)

    def get_topo(self):
        """
        
        """
        topo, _ = etopo(
            self.lon_IMERG, self.lat_IMERG,
            Path(Path(os.getcwd()).parent, 'EarthData'),
            engine='scipy',
        )
        return topo
    
    def download(
            self, path,
    ):
        """
        ***Beware***, it may change over time.
        """
        start_dt = datetime.datetime.strftime(self.start_date, '%Y-%m-%d')
        end_dt = datetime.datetime.strftime(self.end_date, '%Y-%m-%d')
        dts = np.array(
            [
                start_dt + datetime.timedelta(days=day)
                for day in range((end_dt - start_dt).days)
            ]
        )
        years = np.array([_.year for _ in dts])
        products = ['Early', 'Late', 'Final']
        freq = '30T'
        for product in products:
            print(f'Downloading - {product} - ...')
            save_path = Path(path, f'temp_{product}_{freq}')
            if not os.path.exists(save_path):
                os.mkdir(save_path)
            for year in np.unique(years):
                s_dt = np.min(dts[years == year])
                e_dt = np.max(dts[years == year])
                save_path_ = Path(save_path, str(year))
                if not os.path.exists(save_path_):
                    os.mkdir(save_path_)
                download = Download_IMERG(
                    freq,
                    f'{product}',
                    self.bbox, # lon1, lon2, lat1, lat2
                    save_path_,
                )
                download.download(
                    f'{year}-{s_dt.month}-{s_dt.day}',
                    f'{year}-{e_dt.month}-{e_dt.day}',
                )
    
    def create_file_part(
            self, path, file_paths, time_index,
            time_size, t, product, start_end_hour,
    ):
        """
        
        
        """
        def write_dataset(hdf_file, dataset, ds):
            hdf_file.create_dataset(name=dataset, data=ds)
            hdf_file[dataset].attrs['missing_value'] = 'NaN'
            hdf_file[dataset].attrs['missing_value_type'] = 'numpy.nan'
            hdf_file[dataset].attrs['dimension_names'] = ['time', 'lon', 'lat']
            hdf_file[dataset].attrs['dimension_coordinates'] = [0, 1, 2]
            hdf_file[dataset].attrs['original_name'] = dataset
            hdf_file[dataset].attrs['full_name'] = f'{dataset} 24 hours accumulated'
            hdf_file[dataset].attrs['units'] = 'mm'
        if not os.path.exists(path):
            print(path)
            time_index_ = time_index + time_size
            if time_index_ > t.shape[0]:
                time_index_ = t.shape[0]
                precipitationCal, precipitationUncal, IRprecipitation, HQprecipitation = get_imerg_daily_sum(file_paths[time_index:], self.bbox)
            else:
                precipitationCal, precipitationUncal, IRprecipitation, HQprecipitation = get_imerg_daily_sum(file_paths[time_index:time_index_], self.bbox)
            with h5py.File(path, 'w') as hdf_file:
                """File Information"""
                hdf_file.attrs['file_author'] = 'Segura Ellis, Joaquin Sebastian'
                hdf_file.attrs['contact_mail'] = 'joaquin.segura.ellis@mi.unc.edu.ar'
                hdf_file.attrs['creation_time'] = str(datetime.datetime.now(tz=datetime.timezone.utc))
                hdf_file.attrs['description'] = f'This file is based on IMERG {product} half-hourly datasets. It is a daily accumulated precipitation dataset. Every single value has 3 coordinates, the time represents an accumulation from 24 hours before, the longitude and the latitude.'
                hdf_file.attrs['start_end_hour_UTC'] = start_end_hour
                """precipitationCal dataset and its information"""
                write_dataset(hdf_file, 'precipitationCal', precipitationCal)
                """precipitationUncal dataset and its information"""
                # write_dataset(hdf_file, 'precipitationUncal', precipitationUncal)
                """IRprecipitation dataset and its information"""
                # write_dataset(hdf_file, 'IRprecipitation', IRprecipitation)
                """HQprecipitation dataset and its information"""
                # write_dataset(hdf_file, 'HQprecipitation', HQprecipitation)
                """longitude dataset and its information"""
                hdf_file.create_dataset(name='/lon', data=self.lon_IMERG)
                hdf_file['lon'].attrs['coordinate'] = 1
                hdf_file['lon'].attrs['axis'] = 'X'
                hdf_file['lon'].attrs['bounds'] = self.lon_IMERG[[0, -1]]
                hdf_file['lon'].attrs['standard_name'] = 'longitude'
                hdf_file['lon'].attrs['long_name'] = 'Longitude at the center of a 0.10 degree grid'
                hdf_file['lon'].attrs['units'] = 'degrees_east'
                """latitude dataset and its information"""
                hdf_file.create_dataset(name='/lat', data=self.lat_IMERG)
                hdf_file['lat'].attrs['coordinate'] = 2
                hdf_file['lat'].attrs['axis'] = 'Y'
                hdf_file['lat'].attrs['bounds'] = self.lat_IMERG[[0, -1]]
                hdf_file['lat'].attrs['standard_name'] = 'latitude'
                hdf_file['lat'].attrs['long_name'] = 'Latitude at the center of a 0.10 degree grid'
                hdf_file['lat'].attrs['units'] = 'degrees_north'
                """time dataset and its information"""
                hdf_file.create_dataset(name='/time', data=np.arange(0, t.shape[0])[time_index:time_index_])
                hdf_file['time'].attrs['coordinate'] = 0
                hdf_file['time'].attrs['axis'] = 'T'
                hdf_file['time'].attrs['bounds'] = [time_index, time_index_ - 1]
                hdf_file['time'].attrs['calendar'] = 'julian'
                hdf_file['time'].attrs['standard_name'] = 'time'
                hdf_file['time'].attrs['long_name'] = 'Representative time of data'
                hdf_file['time'].attrs['description'] = 'Data is accumulated between this time and 24 hours before'
                hdf_file['time'].attrs['units'] = f'days since {str(t[0])}'
                hdf_file['time'].attrs['first_time'] = str(t[0])
                hdf_file['time'].attrs['format_Python_datetime_library'] = '%Y:%m:%d %H:%M'
                hdf_file['time'].attrs['time_zone'] = 'UTC'

    def gridded_daily_rainfall(
            self, data_path, save_path, start_end_hour, product,
    ):
        """
        
        """
        file_paths, t = np.array(
            [
                [
                    Path(data_path, year, file),
                    datetime.datetime.strptime(
                        file[:file.index('.')], '%Y%m%d%H%M',
                    ),
                ]
                for year in os.listdir(data_path)
                for file in os.listdir(Path(data_path, year))
            ]
        ).T
        first_time = datetime.datetime(
            t[0].year, t[0].month, t[0].day, start_end_hour, 30,
        )
        file_paths = file_paths[t >= first_time]
        file_paths = np.array([
            file_paths[i-48:i]
            for i in range(48, file_paths.shape[0], 48)
        ])
        t = t[t >= first_time]
        t = np.array([t[i] for i in range(47, t.shape[0], 48)])
        time_size = 300
        part = 1
        if not os.path.exists(Path(save_path, f'IMERG_{product}_{start_end_hour}')):
            os.mkdir(Path(save_path, f'IMERG_{product}_{start_end_hour}'))
        for time_index in range(0, t.shape[0], time_size):
            self.create_file_part(
                Path(
                    save_path,
                    f'IMERG_{product}_{start_end_hour}',
                    f'IMERG_{product}_{start_end_hour}_part_{part}.hdf5',
                ),
                file_paths, time_index, time_size, t, product, start_end_hour,
            )
            part += 1

    def get_station(
            self, file,
    ):
        """
        
        """
        ind_k = 'Y_rx1day'
        name = file[:file.find('.')]
        station = Rain_Gauge()
        station.load(Path(self.stations_dir, file))
        station.file = name
        station.period = self.period
        station.result = dict()
        station.data = station.data[station.time < self.end_date]
        station.time = station.time[station.time < self.end_date]
        station.data = station.data[station.time >= self.start_date]
        station.time = station.time[station.time >= self.start_date]
        if station.time.shape[0] > 0:
            station.rxDday_calc(1)
            bbox_condition = inside_bbox(
                self.bbox, station.lat, station.lon,
            )
            min_condition = minimum_lenght(
                station.result[ind_k]['data'], 14,
            )
            elev_condition = station.elevation < 3000
            if bbox_condition and min_condition and elev_condition:
                station.sorted_max_calc()
                station = frequency_analysis(station, replace=1)
                return station
        
    def get_imerg(
            self, station, imerg, idw,
    ):
        """
        
        """
        imerg_data = imerg.data
        imerg_data = idw.interp(imerg_data, axes=[0, 2, 1])
        imerg_dt, index1, index2 = np.intersect1d(
            imerg.time, station.time, return_indices=True,
        )
        imerg_data = imerg_data[index1]
        imerg_data[np.isnan(station.data[index2])] = np.nan
        imerg = Rainfall_Indices(
            imerg_dt, imerg_data, start_month=station.start_month,
        )
        imerg.rxDday_calc(1)
        imerg.sorted_max_calc()
        imerg = frequency_analysis(imerg, replace=1)
        return imerg
    
    def get_analysis(
            self, file, imerg,
    ):
        """
        
        """
        station = self.get_station(file)
        if station is not None:
            idw = IDW_Grid_Interpolation(
                self.lon_IMERG, self.lat_IMERG,
                station.lon, station.lat, 2,
            )
            imerg_e = self.get_imerg(
                station, imerg['Early'][int(station.record_time[:2])],
                idw,
            )
            imerg_l = self.get_imerg(
                station, imerg['Late'][int(station.record_time[:2])],
                idw,
            )
            idw = IDW_Grid_Interpolation(
                self.lon_IMERG, self.lat_IMERG,
                station.lon, station.lat, 3,
            )
            imerg_f = self.get_imerg(
                station, imerg['Final'][int(station.record_time[:2])],
                idw,
            )
            return station, imerg_e, imerg_l, imerg_f

    def create_card(
            self, station, save_path, imerg_e, imerg_l, imerg_f,
    ):
        """
        
        """
        ind_k = 'Y_rx1day'
        card = Card_IMERG([-58, -76, -20, -53], 10, station, save_path)
        if not os.path.exists(Path(card.save_path, f'{card.station.file}.png')):
            card.create_figure(
                title=f'{card.station.file} - {card.station.province}',
                figsize=(19.2, 10.8), dpi=100,
            )
            card.add_imerg(imerg_e, imerg_l, imerg_f)
            card.draw_map1(card.ax_map)
            card.draw_plot1(card.ax_plot1)
            card.draw_plot2(
                card.ax_plot2,
                card.station.result['Y_3_sorted_rx1day']['time'],
                card.station.result[ind_k]['outliers'],
            )
            card.draw_plot3(card.ax_plot3)
            card.draw_plot4(card.ax_plot4, self.T)
            card.draw_table1(
                card.ax_info1,
                card.station.result[ind_k]['data_outliers_tests'],
                card.station.result[ind_k]['outliers'],
            )
            card.add_to_table1(self.T)
            card.fig.tight_layout()
            card.fig.autofmt_xdate()
            plt.savefig(Path(card.save_path, f'{card.station.file}.png'))
            plt.close()
    
    def fill_interp_imerg(
            self, data, mask=None,
    ):
        """
        
        """
        if mask is not None:
            data[mask] = np.nan
        X, Y = np.meshgrid(self.lon_IMERG, self.lat_IMERG)
        points = np.array([X.flatten(), Y.flatten()]).T
        data = data.flatten()
        points = points[~np.isnan(data)]
        data = data[~np.isnan(data)]
        return scipy.interpolate.griddata(
            points, data, (X, Y), method='linear',
        )

    def get_results(
            self, argentina_shp_path,
    ):
        """
        
        """
        ind_k = 'Y_rx1day'
        wo_k = 'data_without_outliers'
        ot_k = 'data_outliers_tests'
        T = [2, 5, 10, 25, 50]
        elevation_mask = self.topo >= 3000
        station_i = np.arange(len(self.stations))[
            np.array([_[0].name for _ in self.stations]) == 'Esquel Aero'
        ][0]
        tests_examples(
            Path(self.outputs_dir, '01_tests_examples.png'),
        )
        map_station_institution(
            self.bbox,
            np.array([_[0].lon for _ in self.stations]),
            np.array([_[0].lat for _ in self.stations]),
            np.array([_[0].institution for _ in self.stations]),
            Path(self.outputs_dir, '01_institutions.png'),
        )
        hist_information(
            np.array([_[0].institution for _ in self.stations]),
            np.array([_[0].lon for _ in self.stations]),
            np.array([_[0].lat for _ in self.stations]),
            np.array([_[0].elevation for _ in self.stations]),
            self.for_use_w_out,
            Path(self.outputs_dir, '01_information.png'),
        )    
        map_start_month(
            self.bbox,
            np.array([_[0].lon for _ in self.stations]),
            np.array([_[0].lat for _ in self.stations]),
            np.array([_[0].start_month for _ in self.stations]),
            Path(self.outputs_dir, '02_start_month.png'),
        )
        interpolation_method_comparison(
            Path(self.outputs_dir, 'interpolations.csv'),
            Path(self.outputs_dir, '04_interpolation_comparison.png'),
        )
        scatter_station_imerg(
            self.stations[station_i][0].data,
            self.stations[station_i][1].data,
            self.stations[station_i][2].data,
            self.stations[station_i][3].data,
            self.stations[station_i][0].name,
            Path(self.outputs_dir, '05_example_daily_data.png'),
        )
        scatter_station_imerg(
            self.stations[station_i][0].result[ind_k]['outliers'][wo_k][-1],
            self.stations[station_i][1].result[ind_k]['outliers'][wo_k][-1],
            self.stations[station_i][2].result[ind_k]['outliers'][wo_k][-1],
            self.stations[station_i][3].result[ind_k]['outliers'][wo_k][-1],
            self.stations[station_i][0].name,
            Path(self.outputs_dir, '06_example_rx1day.png'),
        )
        map_station_test(
            self.bbox, np.array([_[0].lon for _ in self.stations]), np.array([_[0].lat for _ in self.stations]),
            Path(self.outputs_dir, '07_station_tests.png'),
            np.array([
                np.array(self.test_w_out, dtype=bool)[:, 0],
                np.array([
                    minimum_lenght(_[0].result[ind_k]['outliers'][wo_k], 14)
                    for _ in self.stations
                ]),
                np.array([
                    _[0].result[ind_k][ot_k]['iWW']['index']
                    for _ in self.stations
                ]) != 3,
                np.array([
                    _[0].result[ind_k][ot_k]['tMK']['index'][0]
                    for _ in self.stations
                ]) == 1,
                np.array([
                    _[0].result[ind_k][ot_k]['tMK']['index'][0]
                    for _ in self.stations
                ]) == 3,
                np.array([
                    int(_[0].result[ind_k][ot_k]['hPE']['index'])
                    for _ in self.stations
                ]) != 3,
            ]),
        )
        for t in T:
            map_prec(
                self.bbox, self.lon_IMERG, self.lat_IMERG,
                np.array([_[0].lon for _ in self.stations])[self.for_use_w_out],
                np.array([_[0].lat for _ in self.stations])[self.for_use_w_out],
                np.array([
                    self.stations[j][0].result[ind_k]['lognorm'].ppf([t])[1]
                        for j in range(len(self.stations)) if self.for_use_w_out[j]
                ]),
                Path(self.outputs_dir, f'09_stations_{t}_years.png'),
                shp_path=argentina_shp_path, elevation_mask=elevation_mask,
            )
            map_Catalini_comp(
                self.bbox, self.lon_IMERG, self.lat_IMERG,
                np.array([_[0].lon for _ in self.stations])[self.for_use_w_out],
                np.array([_[0].lat for _ in self.stations])[self.for_use_w_out],
                np.array([
                    self.stations[j][0].result[ind_k]['lognorm'].ppf([t])[1]
                        for j in range(len(self.stations)) if self.for_use_w_out[j]
                ]),
                Path(self.outputs_dir, f'14_catalini_comp_{t}_years.png'),
                f'C:\\Users\\joako\\OneDrive\\1 - Posgrado\\Workspace\\IMERG\\arg_data_img\\pmd{t}.tif',
                Path(Path(os.getcwd()).parent, 'IMERG', 'arg_data_img', 'Estaciones', 'Estaciones.shp'),
                shp_path=argentina_shp_path, elevation_mask=elevation_mask,
            )
        bounds = [
            None,
            np.array([
                0, 1, 2, 5, 10, 15, 20, 25,
                30, 35, 40, 45, 50, 60, 70,
            ]),
            np.array([
                0, 1, 2, 5, 10, 15, 20, 25,
                30, 35, 40, 45, 50, 60, 70,
            ]),
            np.array([
                0, 50, 100, 150, 200, 250, 300, 350,
                400, 450, 500, 550, 600, 700, 800,
            ]),
            np.array([
                0.20, 0.24, 0.28, 0.32, 0.36, 0.40, 0.44, 0.48,
                0.52, 0.56, 0.60, 0.70, 0.80, 1.00, 2.00,
            ]),
        ]
        cb_label = [
            'Precipitación (mm/día)', 'Precipitación (mm/día)',
            '', 'Precipitación (mm/día)', '',
        ]
        for i, key in enumerate(['data_mean', 'data_std', 'phi_pmp', 'pmp']):
            map_prec(
                self.bbox, self.lon_IMERG, self.lat_IMERG,
                np.array([_[0].lon for _ in self.stations])[self.for_use],
                np.array([_[0].lat for _ in self.stations])[self.for_use],
                np.array([
                    self.stations[j][0].result[ind_k][key]
                        for j in range(len(self.stations)) if self.for_use[j]
                ]),
                Path(self.outputs_dir, f'12_stations_{key}.png'),
                bounds=bounds[i],
                shp_path=argentina_shp_path, elevation_mask=elevation_mask,
                cb_label=cb_label[i],
            )
        map_prec(
            self.bbox, self.lon_IMERG, self.lat_IMERG,
            np.array([_[0].lon for _ in self.stations])[self.for_use],
            np.array([_[0].lat for _ in self.stations])[self.for_use],
            np.array([
                self.stations[j][0].result[ind_k]['data_std'] / \
                self.stations[j][0].result[ind_k]['data_mean']
                for j in range(len(self.stations))
                if self.for_use[j]
            ]),
            Path(self.outputs_dir, '12_stations_data_cv.png'),
            bounds=bounds[-1],
            shp_path=argentina_shp_path, elevation_mask=elevation_mask,
            cb_label=cb_label[-1],
        )
        for p in ['Early', 'Late', 'Final']:
            rt = 12
            map_imerg_start_month(
                self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat,
                self.imerg[p][rt].start_month.T,
                Path(self.outputs_dir, f'03_imerg_start_month_{p}_{rt}.png'),
                argentina_shp_path, elevation_mask,
            )
            tests = self.imerg[p][rt].result[ind_k]['tests_data_without_outliers']
            cond_iWW = tests['iWW']['index'].T == 3
            cond_hPE = tests['hPE']['index'].T == '3'
            cond_tMK = tests['tMK']['index'][:, :, 0].T == 2
            cond = (cond_iWW * cond_hPE * cond_tMK).astype(bool)
            map_imerg_test(
                self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat,
                cond_iWW, cond_tMK, cond_hPE,
                Path(self.outputs_dir, f'08_imerg_tests_{p}_{rt}.png'),
                argentina_shp_path, elevation_mask,
            )
            for i, t in enumerate(T):
                prec = self.imerg[p][rt].result[ind_k]['lognorm'][:, :, 1, i].T
                if t == 10 and p == 'Final':
                    map_imerg_prec(
                        self.bbox, self.imerg[p][rt].lon,
                        self.imerg[p][rt].lat,
                        np.where(cond, prec, 0),
                        Path(
                            self.outputs_dir,
                            f'10_imerg_prec_{p}_{rt}_{t}_incomplete.png',
                        ),
                        shp_path=argentina_shp_path,
                        elevation_mask=elevation_mask,
                    )
                    map_imerg_prec(
                        self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat, prec,
                        Path(
                            self.outputs_dir,
                            f'10_imerg_prec_{p}_{rt}_{t}_complete.png',
                        ),
                        shp_path=argentina_shp_path,
                        elevation_mask=elevation_mask,
                    )
                map_imerg_prec(
                    self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat,
                    self.fill_interp_imerg(prec, ~cond),
                    Path(
                        self.outputs_dir,
                        f'11_imerg_prec_{p}_{rt}_{t}_years.png',
                    ),
                    shp_path=argentina_shp_path,
                    elevation_mask=elevation_mask,
                )
            tests = self.imerg[p][rt].result[ind_k]['tests_data']
            cond_iWW = tests['iWW']['index'].T == 3
            cond_hPE = tests['hPE']['index'].T == '3'
            cond_tMK = tests['tMK']['index'][:, :, 0].T == 2
            cond = (cond_iWW * cond_hPE * cond_tMK).astype(bool)
            for i, key in enumerate(['mean', 'std', 'phi_pmp', 'pmp']):
                map_imerg_prec(
                    self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat,
                    self.fill_interp_imerg(
                        self.imerg[p][rt].result[ind_k][key].T, ~cond,
                    ),
                    Path(self.outputs_dir, f'13_imerg_prec_{p}_{rt}_{key}.png'),
                    bounds=bounds[i],
                    shp_path=argentina_shp_path,
                    elevation_mask=elevation_mask,
                    cb_label=cb_label[i],
                )
            map_imerg_prec(
                self.bbox, self.imerg[p][rt].lon, self.imerg[p][rt].lat,
                self.fill_interp_imerg(
                    self.imerg[p][rt].result[ind_k]['std'].T / \
                    self.imerg[p][rt].result[ind_k]['mean'].T,
                    ~cond,
                ),
                Path(self.outputs_dir, f'13_imerg_prec_{p}_{rt}_cv.png'),
                bounds=bounds[-1],
                shp_path=argentina_shp_path,
                elevation_mask=elevation_mask,
                cb_label=cb_label[-1],
            )
        conf_int(
            np.array([
                self.stations[j][0].result[ind_k]['lognorm'].ppf(T)
                for j in range(len(self.stations))
                if self.for_use_w_out[j]
            ]),
            Path(self.outputs_dir, f'14_conf_int.png'),
        )
        map_error(
            self.bbox,
            np.array(
                [_[0].lon for _ in self.stations]
            )[self.for_use_w_out],
            np.array(
                [_[0].lat for _ in self.stations]
            )[self.for_use_w_out],
            np.array(
                [_[0].institution for _ in self.stations]
            )[self.for_use_w_out],
            self.test_w_out[self.for_use_w_out],
            self.error[self.for_use_w_out],
            Path(self.outputs_dir, '15_mape_imerg.png'),
        )
        comp_result(
            self.stations, self.for_use_w_out, self.test_w_out,
            Path(self.outputs_dir, '15_comp_imerg.png'),
        )
        comp_variables(
            np.array(
                [_[0].institution for _ in self.stations]
            )[self.for_use_w_out],
            np.array(
                [_[0].lon for _ in self.stations]
            )[self.for_use_w_out],
            np.array(
                [_[0].lat for _ in self.stations]
            )[self.for_use_w_out],
            np.array(
                [_[0].elevation for _ in self.stations]
            )[self.for_use_w_out],
            self.test_w_out[self.for_use_w_out],
            self.error[self.for_use_w_out],
            Path(self.outputs_dir, '15_comp_vars_imerg.png'),
        )
    
    def save_results(self):
        """
        
        """
        ind_k = 'Y_rx1day'
        for p in ['Early', 'Late', 'Final']:
            save_path = Path(self.outputs_dir, f'imerg_{p}.risa')
            if not os.path.exists(save_path):
                data_dict = {
                    'lon': program.lon_IMERG,
                    'lat': program.lat_IMERG,
                    'T': program.T,
                }
                tests = program.imerg[p][12].result[ind_k]['tests_data']
                cond_iWW = tests['iWW']['index'].T == 3
                cond_hPE = tests['hPE']['index'].T == '3'
                cond_tMK = tests['tMK']['index'][:, :, 0].T == 2
                data_dict[f'{p}_tests'] = (cond_iWW * cond_hPE * cond_tMK).astype(bool)
                data_dict[p] = program.imerg[p][12].result[ind_k]['lognorm']
                with open(save_path, 'wb') as f:
                    pickle.dump(data_dict, f)

    def execute(
            self, stations_dir, imerg_dir, rg_bin_dir, imerg_bin_dir,
    ):
        """
        
        """
        ind_k = 'Y_rx1day'
        logn_k = 'lognorm'
        out_k = 'outliers'
        wo_k = 'data_without_outliers'
        self.create_paths(stations_dir, rg_bin_dir, imerg_bin_dir)
        self.topo = bin_file(Path(self.bin_dir, 'elev.topo'), self.get_topo)
        self.imerg = {
            p: {
                h: get_imerg_grid(
                    Path(imerg_dir, f'IMERG_{p}_{h}'),
                    Path(self.imerg_bin_dir, f'IMERG_{p}_{h}.pickle'),
                    self.T,
                    dt_lims=[self.start_date, self.end_date],
                    save=True, analyze=True,
                ) for h in [3, 12]
            } for p in ['Early', 'Late', 'Final']
        }
        for file in tqdm(os.listdir(self.stations_dir)):
            print(file)
            name = file[:file.find('.')]
            bin_ = bin_file(
                Path(self.rg_bin_dir, f'{name}.pickle'),
                self.get_analysis, [file, self.imerg],
            )
            if bin_ is not None:
                station, imerg_e, imerg_l, imerg_f = bin_
                self.stations.append([station, imerg_e, imerg_l, imerg_f])
                test_w_out = [
                    freq_conditions(station), freq_conditions(imerg_e),
                    freq_conditions(imerg_l), freq_conditions(imerg_f),
                ]
                self.for_use_w_out.append(
                    test_w_out[0] and (
                        test_w_out[1] or test_w_out[2] or test_w_out[3]
                    )
                )
                self.test_w_out.append(test_w_out)
                test = [
                    freq_conditions(station, w_out=False),
                    freq_conditions(imerg_e, w_out=False),
                    freq_conditions(imerg_l, w_out=False),
                    freq_conditions(imerg_f, w_out=False),
                ]
                self.for_use.append(
                    test[0] and (test[1] or test[2] or test[3])
                )
                self.test.append(test)
                self.error.append([
                    Card_IMERG.error(
                        station.result[ind_k][logn_k].ppf(self.T)[1],
                        imerg_e.result[ind_k][logn_k].ppf(self.T)[1],
                    ),
                    Card_IMERG.error(
                        station.result[ind_k][logn_k].ppf(self.T)[1],
                        imerg_l.result[ind_k][logn_k].ppf(self.T)[1],
                    ),
                    Card_IMERG.error(
                        station.result[ind_k][logn_k].ppf(self.T)[1],
                        imerg_f.result[ind_k][logn_k].ppf(self.T)[1],
                    ),
                ])
                if not test_w_out[1]:
                    imerg_e.result[ind_k][out_k][wo_k][-1][:] = np.nan
                if not test_w_out[2]:
                    imerg_l.result[ind_k][out_k][wo_k][-1][:] = np.nan
                if not test_w_out[3]:
                    imerg_f.result[ind_k][out_k][wo_k][-1][:] = np.nan
                if not test[1]:
                    imerg_e.result[ind_k]['data'][:] = np.nan
                if not test[2]:
                    imerg_l.result[ind_k]['data'][:] = np.nan
                if not test[3]:
                    imerg_f.result[ind_k]['data'][:] = np.nan
                self.create_card(
                    station, self.card_dir, imerg_e, imerg_l, imerg_f,
                )
            clear_output()
        self.test_w_out = np.array(self.test_w_out)
        self.test = np.array(self.test)
        self.for_use_w_out = np.array(self.for_use_w_out, dtype=bool)
        self.for_use = np.array(self.for_use, dtype=bool)
        self.error = np.array(self.error)
        self.get_results(
            Path(
                Path(os.getcwd()).parent, 'IGN_data',
                'shp', 'argentina', 'pais',
            )
        )
        self.save_results()

if __name__=='__main__':
    print('Main')
    """
    Some directories paths.
    """
    stations_dir = Path(
        os.getcwd(), 'inputs', 'rain_gauge', 'preprocess_csv',
    )
    imerg_dir = Path(os.getcwd(), 'inputs', 'gridded')
    rg_bin_dir = 'rain_gauge'
    imerg_bin_dir = 'imerg_grid_analyze'
    """
    Run preprocess of rain gauges.
    """
    # run_preprocess(Path('rain_gauge'))
    """
    Main object.
    """
    program = Main()
    """
    Command to download IMERG half-hourly data.
    ***Becareful, it takes around 2 months***
    """
    # program.download(Path('f:'))
    """
    The following commands are made to create the daily files of IMERG
    half-hourly data.
    ***Becareful, a lot of storage needed.***
    """
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Early_30T'), Path('m:'), 3, 'Early',
    # )
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Early_30T'), Path('m:'), 12, 'Early',
    # )
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Late_30T'), Path('m:'), 3, 'Late',
    # )
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Late_30T'), Path('m:'), 12, 'Late',
    # )
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Final_30T'), Path('m:'), 3, 'Final',
    # )
    # program.gridded_daily_rainfall(
    #     Path('f:', 'temp_Final_30T'), Path('m:'), 12, 'Final',
    # )
    """
    Execution of the program.
    ***Becareful, it takes around 12 hours***
    """
    program.execute(
        stations_dir, imerg_dir, rg_bin_dir, imerg_bin_dir
    )

In [ ]:
get_design_rainfall(
    'Early', T=10, lon=-60, lat=-35,
)

In [ ]:
def dropnan(data):
    """
    
    """
    return data[~np.isnan(data)]

def outliers_result(stations):
    """
    
    """
    ind_k = 'Y_rx1day'
    wo_k = 'data_without_outliers'
    low = np.array([[dropnan(__.result[ind_k]['outliers']['low_outliers']).shape[0] for __ in _] for _ in stations], dtype=float)
    high = np.array([[dropnan(__.result[ind_k]['outliers']['high_outliers']).shape[0] for __ in _] for _ in stations], dtype=float)
    print('Number of low outliers')
    print(np.array([[np.sum(low[:, j] * low[:, i]) for i in range(4)] for j in range(4)]))
    print('Number of high outliers')
    print(np.array([[np.sum(high[:, j] * high[:, i]) for i in range(4)] for j in range(4)]))
    aux = np.array([[dropnan(__.result[ind_k]['data']).shape[0] for __ in _] for _ in stations]) == np.array([[dropnan(__.result[ind_k]['outliers'][wo_k][-1]).shape[0] for __ in _] for _ in stations])
    print('Number of replacements:', np.sum(aux * high * ~low.astype(bool)))

def tests_result(obj):
    """
    
    """
    print('For use:', np.sum(obj.for_use))
    print('For use without outliers:', np.sum(obj.for_use_w_out))
    institution = np.array([_[0].institution for _ in obj.stations])
    for inst in np.unique(institution):
        print(
            inst, '\t',
            np.sum(institution[~obj.for_use_w_out] == inst),
            '\t', np.sum(institution == inst),
            '\t', 100 * np.sum(institution[~obj.for_use_w_out] == inst) / np.sum(institution == inst),
        )

outliers_result(program.stations)
tests_result(program)